In [1]:
from keras.optimizers import SGD
from convnetskeras.convnets import preprocess_image_batch, convnet
from convnetskeras.imagenet_tool import synset_to_dfs_ids
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tq

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [2]:
import os
from fnmatch import fnmatch

l = []

root = './train/'
pattern = "*.jpg"

for path, subdirs, files in os.walk(root):
    for name in files:
        if fnmatch(name, pattern):
            l.append(os.path.join(path, name))

In [3]:
l[0]

'./train/ALB\\img_00003.jpg'

In [ ]:
s = 7
dest = [ string[:s] + "_heatmap" + string[s:] for string in l]
dest[0]

'./train_heatmap/ALB\\img_00003.jpg'

In [ ]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model = convnet('vgg_19',weights_path="vgg19_weights.h5", heatmap=True)
model.compile(optimizer=sgd, loss='mse')

for i in tq(range(len(l))):
    
    im = preprocess_image_batch([l[i]], color_mode="bgr")

    out = model.predict(im)

    s = "n02512053"
    ids = synset_to_dfs_ids(s)
    heatmap = out[0,ids].sum(axis=0)

    # Then, we can get the image
    import matplotlib.pyplot as plt
    plt.imsave(dest[i],heatmap)
    